# Part 2: Plotting and Basic Spatial Metrics

**Instructor:** Tim Treis
**Time:** 35 minutes

---

Now that we understand what a `SpatialData` object is and how to explore it interactively, let's learn how to create static, publication-quality plots and ask our first spatial question: "Is this gene's expression pattern random, or is it spatially organized?"

**Goals:**
1. Use `spatialdata-plot` to create layered, static images.
2. Introduce a different technology: spot-based Visium data.
3. Use `squidpy` to calculate and visualize a key spatial statistic, Moran's I.

### Setup
First, let's import the libraries we'll need for this notebook.

In [ ]:
%load_ext jupyter_black

import spatialdata as sd
import spatialdata_plot as sdp
import matplotlib.pyplot as plt
import scanpy as sc
import squidpy as sq

data_path = "../data/"

### Static Plotting with `spatialdata-plot`

For this section, we will use a dataset from a 10x Genomics Visium experiment. Unlike the high-resolution Xenium data from the previous notebook, Visium is a **spot-based** technology. It captures the whole transcriptome, but at the resolution of spots (~55µm) which may contain multiple cells.

![Visium technology](../resources/visium_tech.png)

Let's load our pre-processed Visium dataset.

In [ ]:
sdata_visium = sd.read_zarr(data_path + "visium_glioblastoma_subset.zarr")
sdata_visium

The `spatialdata-plot` library adds a `.pl` accessor to our `SpatialData` object. We can chain `render_*` functions to build up a plot layer by layer, similar to `ggplot`.

Let's start by rendering the histology image.

In [ ]:
sdata_visium.pl.render_images().pl.show(coordinate_systems="downscaled_hires")

Now, let's overlay the circular Visium spots (`Shapes`) on top of the image. We can make them semi-transparent to see the tissue underneath.

In [ ]:
(
    sdata_visium.pl.render_images()
    .pl.render_shapes(fill_alpha=0.3)
    .pl.show("downscaled_hires", figsize=(8, 8))
)

### Identifying Spatially Variable Genes

Simply seeing the spots isn't very informative. A common first question in spatial analysis is to identify genes that show a non-random spatial pattern. Are some genes expressed only in one corner of the tissue? 

To answer this, we can use tools from `squidpy` to calculate a spatial autocorrelation score called **Moran's I**.

![spatial patterns](../resources/spatial_patterns.png)

*   A **high positive** Moran's I (near +1) means the gene is **clustered** spatially.
*   A Moran's I **near 0** means the gene expression is **random**.
*   A **high negative** Moran's I (near -1) means the gene shows a **dispersed** or checkerboard-like pattern.

To calculate Moran's I, we first need to build a **spatial graph**. This graph connects spots that are neighbors in physical space, which allows `squidpy` to compare their expression values.

In [ ]:
# Get the AnnData table from our SpatialData object
adata_visium = sdata_visium.tables["table"]

# Build the spatial neighborhood graph
sq.gr.spatial_neighbors(adata_visium)

# Calculate Moran's I for all genes
sq.gr.spatial_autocorr(
    adata_visium,
    mode="moran",
    n_perms=100,  # Use more permutations for a real analysis
    n_jobs=4,  # Use multiple cores to speed it up
)

# Let's look at the genes with the highest Moran's I score (most clustered)
adata_visium.uns["moranI"].head(5)

Now we can use `spatialdata-plot` to visualize the expression of one of these top-scoring genes. We expect to see a clear spatial pattern.

Let's plot the expression of `CST3`, which has a very high score.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Plot the H&E image on the left
sdata_visium.pl.render_images().pl.show(
    "downscaled_hires", ax=axs[0], title="H&E Image"
)

# Plot the gene expression on the right by coloring the spots
(
    sdata_visium.pl.render_images()
    .pl.render_shapes(color="CST3", size=250)  # Color spots by CST3 expression
    .pl.show(
        "downscaled_hires", ax=axs[1], title="CST3 Expression (Spatially Variable)"
    )
)

As we can see, the gene expression is highly concentrated in a specific region of the tissue, confirming the high Moran's I score.

This concludes the first part of our workshop. We've learned how to load and interact with different spatial technologies and how to perform basic plotting and spatial statistical analysis. In the next part, Anthony will guide us through a complete downstream analysis, including cell type clustering and integrating data from multiple experiments.